## Connect to MongoDB

In [2]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os 
from dotenv import load_dotenv

load_dotenv()

uri = os.getenv('uri')

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


## Read Data

In [3]:
db = client["db_products"]
collection = db["products"]

In [4]:
collection.find_one()

{'_id': ObjectId('682f9e6a407ffb562c1739e8'),
 'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'Latitude': -13.29,
 'Longitude': -41.71}

## Rename "lat" and "lon" columns

In [5]:
collection.update_many({}, {"$rename": {"lat": "Latitude", "lon": "Longitude"}} )

In [6]:
collection.find_one()

{'_id': ObjectId('682f9e6a407ffb562c1739e8'),
 'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'Latitude': -13.29,
 'Longitude': -41.71}

## Filter category books log 

In [7]:
collection.distinct("Categoria do Produto")

['brinquedos',
 'eletrodomesticos',
 'eletronicos',
 'esporte e lazer',
 'instrumentos musicais',
 'livros',
 'moveis',
 'utilidades domesticas']

In [8]:
query = {"Categoria do Produto": "livros"}

books_list = []
for doc in collection.find(query):
    books_list.append(doc)

In [9]:
len(books_list)

742

In [10]:
books_list[-1]

{'_id': ObjectId('682f9e6a407ffb562c175ebf'),
 'Produto': 'Dashboards com Power BI',
 'Categoria do Produto': 'livros',
 'Preço': 54.85,
 'Frete': 3.4517020142,
 'Data da Compra': '31/12/2022',
 'Vendedor': 'Lucas Oliveira',
 'Local da compra': 'RJ',
 'Avaliação da compra': 5,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'Latitude': -22.25,
 'Longitude': -42.66}

## Load Data into DataFrame

In [24]:
import pandas as pd

In [25]:
df_books = pd.DataFrame(books_list)

In [26]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   _id                     742 non-null    object 
 1   Produto                 742 non-null    object 
 2   Categoria do Produto    742 non-null    object 
 3   Preço                   742 non-null    float64
 4   Frete                   742 non-null    float64
 5   Data da Compra          742 non-null    object 
 6   Vendedor                742 non-null    object 
 7   Local da compra         742 non-null    object 
 8   Avaliação da compra     742 non-null    int64  
 9   Tipo de pagamento       742 non-null    object 
 10  Quantidade de parcelas  742 non-null    int64  
 11  Latitude                742 non-null    float64
 12  Longitude               742 non-null    float64
dtypes: float64(4), int64(2), object(7)
memory usage: 75.5+ KB


In [23]:
df_books.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [22]:
df_books.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,682f9e6a407ffb562c1739e8,Modelagem preditiva,livros,92.45,5.609697,01/01/2020,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,682f9e6a407ffb562c1739e9,Iniciando em programação,livros,43.84,0.000000,01/01/2020,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,682f9e6a407ffb562c1739fb,Iniciando em programação,livros,63.25,3.894137,01/01/2022,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,682f9e6a407ffb562c173a09,Ciência de dados com python,livros,86.13,5.273176,01/02/2021,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,682f9e6a407ffb562c173a0b,Ciência de dados com python,livros,72.75,1.458158,01/02/2021,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


## Transform and Filter Data

In [45]:
df_books = df_books.rename(columns={"_id": "id",
                         "Produto": "produto",
                         "Categoria do Produto": "categoria_do_produto",
                         "Preço": "preco",
                         "Frete": "frete",
                         "Data da Compra": "data_da_compra",
                         "Vendedor": "vendedor",
                         "Local da compra": "local_da_compra",
                         "Avaliação da compra": "avaliacao_da_compra",
                         "Tipo de pagamento": "tipo_de_pagamento",
                         "Quantidade de parcelas": "quantidade_de_parcelas",
                         "Latitude": "latitude",
                         "Longitude": "longitude"
                         })

In [46]:
df_books.columns

Index(['id', 'produto', 'categoria_do_produto', 'preco', 'frete',
       'data_da_compra', 'vendedor', 'local_da_compra', 'avaliacao_da_compra',
       'tipo_de_pagamento', 'quantidade_de_parcelas', 'latitude', 'longitude'],
      dtype='object')

In [49]:
df_books["data_da_compra"] = pd.to_datetime(df_books["data_da_compra"], format="%d/%m/%Y")
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      742 non-null    object        
 1   produto                 742 non-null    object        
 2   categoria_do_produto    742 non-null    object        
 3   preco                   742 non-null    float64       
 4   frete                   742 non-null    float64       
 5   data_da_compra          742 non-null    datetime64[ns]
 6   vendedor                742 non-null    object        
 7   local_da_compra         742 non-null    object        
 8   avaliacao_da_compra     742 non-null    int64         
 9   tipo_de_pagamento       742 non-null    object        
 10  quantidade_de_parcelas  742 non-null    int64         
 11  latitude                742 non-null    float64       
 12  longitude               742 non-null    float64   

In [50]:
df_books["data_da_compra"] = df_books["data_da_compra"].dt.strftime("%Y-%m-%d")

In [54]:
df_books.tail()

,id,produto,categoria_do_produto,preco,frete,data_da_compra,vendedor,local_da_compra,avaliacao_da_compra,tipo_de_pagamento,quantidade_de_parcelas,latitude,longitude
737,682f9e6a407ffb562c175e83,Dashboards com Power BI,livros,41.27,2.728608,2022-07-31,João Souza,SC,5,cartao_credito,12,-27.45,-50.95
738,682f9e6a407ffb562c175e87,Ciência de dados com python,livros,81.58,4.632666,2020-08-31,Juliana Costa,SP,5,cartao_credito,2,-22.19,-48.79
739,682f9e6a407ffb562c175e8d,Modelagem preditiva,livros,109.17,6.701389,2021-08-31,Felipe Santos,MG,3,cartao_credito,3,-18.10,-44.38
740,682f9e6a407ffb562c175e99,Modelagem preditiva,livros,94.89,5.578872,2022-08-31,Pedro Gomes,SP,5,cartao_credito,1,-22.19,-48.79
741,682f9e6a407ffb562c175ebf,Dashboards com Power BI,livros,54.85,3.451702,2022-12-31,Lucas Oliveira,RJ,5,cartao_credito,3,-22.25,-42.66


## Load data into CSV 

In [56]:
path = "../data/table_books.csv"
df_books.to_csv(path, index=False)

## Filter sold Products from 2021

In [58]:
collection.find_one()

{'_id': ObjectId('682f9e6a407ffb562c1739e8'),
 'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'Latitude': -13.29,
 'Longitude': -41.71}

In [ ]:
query = {"Data da Compra": {"$regex": "/202[1-9]"}}

In [62]:
products_list = []
for doc in collection.find(query):
    products_list.append(doc)

In [63]:
products_list

[{'_id': ObjectId('682f9e6a407ffb562c1739f1'),
  'Produto': 'Xadrez de madeira',
  'Categoria do Produto': 'brinquedos',
  'Preço': 25.23,
  'Frete': 0.0,
  'Data da Compra': '01/01/2021',
  'Vendedor': 'Thiago Silva',
  'Local da compra': 'BA',
  'Avaliação da compra': 5,
  'Tipo de pagamento': 'cartao_credito',
  'Quantidade de parcelas': 2,
  'Latitude': -13.29,
  'Longitude': -41.71},
 {'_id': ObjectId('682f9e6a407ffb562c1739f2'),
  'Produto': 'Impressora',
  'Categoria do Produto': 'eletronicos',
  'Preço': 322.04,
  'Frete': 14.7321002401,
  'Data da Compra': '01/01/2021',
  'Vendedor': 'João Souza',
  'Local da compra': 'SP',
  'Avaliação da compra': 3,
  'Tipo de pagamento': 'cartao_credito',
  'Quantidade de parcelas': 1,
  'Latitude': -22.19,
  'Longitude': -48.79},
 {'_id': ObjectId('682f9e6a407ffb562c1739f3'),
  'Produto': 'Mesa de centro',
  'Categoria do Produto': 'moveis',
  'Preço': 282.22,
  'Frete': 12.6118050347,
  'Data da Compra': '01/01/2021',
  'Vendedor': 'João 